<a href="https://colab.research.google.com/github/vignesh7-byte/vignesh7-byte/blob/main/2303A51640_batch_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

prediction of Gender by Names given to the new born babies

Q1.identify the top-10 names given to the babies

Q2.find the Max and Min count of babies with similar names
                                                                        
Q3.name the gender which as aquired the most names among the babies

Q4.identify the top-5 probabilities of occurance of male and female babies
Q5.name the top-5 names with highest probabilities
Q6.apply either classification of clustering of Regression model and evaluate the accurracy error metrics of the model

In [16]:
import pandas as pd

# ... (Your existing data and DataFrame 'df') ...

# Sort the DataFrame by 'Count' in descending order
df_sorted = df.sort_values(by=['Count'], ascending=False)

# Select the top 10 names
top_10_names = df_sorted.head(10)['Name'].tolist()

# Print the top 10 names
print("Top 10 baby names:")
for name in top_10_names:
    print(name)

Top 10 baby names:
James
John
Robert
Michael
William
Mary
David
Joseph
Richard
Charles


In [24]:
!pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.6 MB/s eta 0:00:00


In [26]:
!pip install fuzzywuzzy python-Levenshtein
import pandas as pd
from fuzzywuzzy import fuzz

# ... (Your existing data and DataFrame 'df') ...

# Assuming your DataFrame only has 'Name' and 'Count' columns
# Modify this if you have other relevant columns for grouping

def group_similar_names(names, threshold=80):
    """
    Groups similar names based on a similarity threshold.

    Args:
        names: A list of tuples containing (name, count).
        threshold: The similarity threshold (0-100).

    Returns:
        A list of grouped names.
    """
    grouped = []
    for name, count in names:  # Changed to iterate over name and count only
        found_group = False
        for group in grouped:
            group_name = group[0][0]
            if fuzz.partial_ratio(name, group_name) >= threshold:
                group.append((name, count))  # Appended name and count only
                found_group = True
                break
        if not found_group:
            grouped.append([(name, count)])  # Initialized with name and count only
    return grouped

# Group the names based on similarity
# Select only the 'Name' and 'Count' columns
grouped_data = group_similar_names(df[['Name', 'Count']].values)

# Calculate the total counts for each group
grouped_counts = [(group[0][0], sum([count for name, count in group])) for group in grouped_data] # Updated to reflect changes in group_similar_names output

# Create a DataFrame
grouped_df = pd.DataFrame(grouped_counts, columns=['Group', 'Total Count'])

# Find the group with max and min total counts
max_count_group = grouped_df.loc[grouped_df['Total Count'].idxmax()]
min_count_group = grouped_df.loc[grouped_df['Total Count'].idxmin()]

print("Max Count Group:")
print(max_count_group)
print("\nMin Count Group:")
print(min_count_group)

Max Count Group:
Group          Aiden
Total Count    18600
Name: 2, dtype: object

Min Count Group:
Group          John
Total Count    8000
Name: 8, dtype: object


In [32]:
import pandas as pd

data = [
    ('Emma', 12345, 'Female'),
    ('Emily', 10120, 'Female'),
    ('Emely', 1200, 'Female'),
    ('Aiden', 8900, 'Male'),
    ('Ayden', 8200, 'Male'),
    ('Aidan', 1500, 'Male'),
    ('Jackson', 10500, 'Male'),
    ('Jaxon', 9400, 'Male'),
    ('Sophia', 15000, 'Female'),
    ('Sofia', 14000, 'Female'),
    ('James', 9000, 'Male'),
    ('John', 8000, 'Male'),
    ('Olivia', 13000, 'Female'),
    ('Liam', 11000, 'Male')
]

# Create a DataFrame from the sample data
df = pd.DataFrame(data, columns=['Name', 'Count', 'Gender'])


In [33]:
# Count the number of unique names for each gender
gender_name_count = df.groupby('Gender')['Name'].nunique()

# Identify the gender with the most unique names
max_gender = gender_name_count.idxmax()

print(f"The gender which has acquired the most names is: {max_gender}")


The gender which has acquired the most names is: Male


In [34]:
# Total count of babies
total_babies = df['Count'].sum()

# Calculate the probability of occurrence of male and female babies
gender_probabilities = df.groupby('Gender')['Count'].sum() / total_babies

# Get the top-5 probabilities
top_5_gender_probs = gender_probabilities.sort_values(ascending=False).head(5)

print("Top-5 Probabilities of Occurrence of Male and Female Babies:")
print(top_5_gender_probs)


Top-5 Probabilities of Occurrence of Male and Female Babies:
Gender
Male      0.503159
Female    0.496841
Name: Count, dtype: float64


In [35]:
# Calculate the probability for each name
df['Name Probability'] = df['Count'] / total_babies

# Get the top-5 names by probability
top_5_names_by_prob = df[['Name', 'Name Probability']].sort_values(by='Name Probability', ascending=False).head(5)

print("Top-5 Names with Highest Probabilities:")
print(top_5_names_by_prob)


Top-5 Names with Highest Probabilities:
      Name  Name Probability
8   Sophia          0.113494
9    Sofia          0.105928
12  Olivia          0.098362
0     Emma          0.093406
13    Liam          0.083229


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Encode the gender labels
le = LabelEncoder()
df['Gender_encoded'] = le.fit_transform(df['Gender'])

# Feature and target
X = df['Name']  # We need to convert the Name into a numerical feature
y = df['Gender_encoded']

# Convert names to a numerical feature (you can use other techniques such as TF-IDF, CountVectorizer, etc.)
# For simplicity, let's use a basic method by taking the length of the name
X = X.apply(lambda x: len(x)).values.reshape(-1, 1)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 40.00%

Classification Report:
              precision    recall  f1-score   support

      Female       0.00      0.00      0.00         3
        Male       0.40      1.00      0.57         2

    accuracy                           0.40         5
   macro avg       0.20      0.50      0.29         5
weighted avg       0.16      0.40      0.23         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
